En este bloque se carga el dataset resultante del proceso de unión (join) y se estandarizan los tipos básicos de las columnas temporales y categóricas, con el fin de garantizar una base consistente para las verificaciones posteriores.

In [1]:
# Block 1 — Load join dataset and set dtypes

import pandas as pd
import numpy as np

PATH = r"C:\Users\wilmerbelza\Documents\Prediction model\mh.csv"  # ajusta si corresponde
dfj = pd.read_csv(PATH)

# Expected datetime columns; coerce errors to NaT
dt_cols = [
    'FechaRegistro', 'FechaTriage', 'FechaPrimeraEvolucion',
    'FechaPrimeraInterconsulta', 'FechaPrimeraEvaluacion', 'FechaEgreso'
]
for c in dt_cols:
    if c in dfj.columns:
        dfj[c] = pd.to_datetime(dfj[c], errors='coerce')

# Categorical / string normalization
for c in ['Genero', 'SalidaClinica', 'MotivoConsulta']:
    if c in dfj.columns:
        dfj[c] = dfj[c].astype(str)

print("=== Loaded join dataset ===")
print("shape:", dfj.shape)
print("columns:", list(dfj.columns))
print(dfj.head(3))

=== Loaded join dataset ===
shape: (46135, 16)
columns: ['id_anon', 'Ingreso', 'Genero', 'EdadAtencion', 'FechaRegistro', 'FechaTriage', 'ClasificacionTriage', 'MotivoConsulta', 'FechaPrimeraEvolucion', 'FechaPrimeraInterconsulta', 'FechaPrimeraEvaluacion', 'FechaEgreso', 'SalidaClinica', 'min_registro_a_triage', 'min_triage_a_eval', 'min_registro_a_eval']
            id_anon  Ingreso     Genero  EdadAtencion       FechaRegistro  \
0  6124828209892124        1   Femenino            27 2023-08-17 18:15:08   
1  9294009b87eac4f1        1  Masculino            26 2023-04-04 12:16:32   
2  63cd4a4d6b899314        2  Masculino            26 2023-09-22 11:16:51   

          FechaTriage  ClasificacionTriage  \
0 2023-08-17 18:31:41                    3   
1 2023-04-04 12:21:24                    2   
2 2023-09-22 11:20:51                    3   

                                      MotivoConsulta FechaPrimeraEvolucion  \
0  Se realiza atención del paciente, con medidas ...   2023-08-17 20:

Se valida que el esquema del join coincida con la estructura esperada (presencia y orden de columnas clave). Cualquier desalineación detona un error temprano y evita continuar con un dataset inconsistente.

In [2]:
# Block 2 — Schema check (presence & minimal order)

expected_cols = [
    'id_anon','Ingreso','Genero','EdadAtencion',
    'FechaRegistro','FechaTriage','ClasificacionTriage','MotivoConsulta',
    'FechaPrimeraEvolucion','FechaPrimeraInterconsulta','FechaPrimeraEvaluacion',
    'FechaEgreso','SalidaClinica',
    'min_registro_a_triage','min_triage_a_eval','min_registro_a_eval'
]

missing = [c for c in expected_cols if c not in dfj.columns]
extra = [c for c in dfj.columns if c not in expected_cols]

print("Missing columns:", missing)
print("Extra columns:", extra)
assert not missing, f"Esquema incompleto. Faltan columnas: {missing}"

# Reordenar para consistencia (no elimina extras)
dfj = dfj[[c for c in expected_cols if c in dfj.columns] + extra]
print("Schema OK. Column order normalized (expected first).")

Missing columns: []
Extra columns: []
Schema OK. Column order normalized (expected first).


Se revisan el tamaño del dataset, los tipos de datos y un resumen descriptivo para confirmar que las variables numéricas y temporales tienen valores plausibles antes de aplicar controles específicos.

In [4]:
# Block 3 — Quick info & describe

print("=== DataFrame info ===")
print(dfj.info())

print("\n=== Descriptive statistics (numeric) ===")
print(dfj.select_dtypes(include=[np.number]).describe().T)

print("\n=== Date columns (non-null counts) ===")
for c in ['FechaRegistro','FechaTriage','FechaPrimeraEvaluacion','FechaPrimeraEvolucion','FechaPrimeraInterconsulta','FechaEgreso']:
    if c in dfj.columns:
        nn = dfj[c].notna().mean()*100
        print(f"{c}: non-null {nn:.2f}%")

=== DataFrame info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46135 entries, 0 to 46134
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id_anon                    46135 non-null  object        
 1   Ingreso                    46135 non-null  int64         
 2   Genero                     46135 non-null  object        
 3   EdadAtencion               46135 non-null  int64         
 4   FechaRegistro              46135 non-null  datetime64[ns]
 5   FechaTriage                46135 non-null  datetime64[ns]
 6   ClasificacionTriage        46135 non-null  int64         
 7   MotivoConsulta             46135 non-null  object        
 8   FechaPrimeraEvolucion      46113 non-null  datetime64[ns]
 9   FechaPrimeraInterconsulta  23639 non-null  datetime64[ns]
 10  FechaPrimeraEvaluacion     46135 non-null  datetime64[ns]
 11  FechaEgreso                46135 non-null  d

Se evalúa la cobertura de campos temporales críticos (registro, triage y primera evaluación) y se reportan ausencias en otros campos de interés (evolución e interconsulta).

In [5]:
# Block 4 — Critical nulls (%)

def null_pct(s): return s.isna().mean()*100

critical = ['FechaRegistro','FechaTriage','FechaPrimeraEvaluacion']
others = ['FechaPrimeraEvolucion','FechaPrimeraInterconsulta']

print("=== Critical nulls (%) ===")
for c in critical:
    if c in dfj.columns:
        print(f" - {c}: {null_pct(dfj[c]):.2f}%")

print("\n=== Other nulls (%) ===")
for c in others:
    if c in dfj.columns:
        print(f" - {c}: {null_pct(dfj[c]):.2f}%")

=== Critical nulls (%) ===
 - FechaRegistro: 0.00%
 - FechaTriage: 0.00%
 - FechaPrimeraEvaluacion: 0.00%

=== Other nulls (%) ===
 - FechaPrimeraEvolucion: 0.05%
 - FechaPrimeraInterconsulta: 48.76%


Se verifica la existencia de episodios duplicados. Un episodio se define por id_anon + Ingreso. En caso de duplicidad, se listan ejemplos para diagnóstico.

In [6]:
# Block 5 — Episode duplicates check

assert 'id_anon' in dfj.columns and 'Ingreso' in dfj.columns, "Faltan id_anon/Ingreso"
dups = dfj.duplicated(subset=['id_anon','Ingreso'], keep=False)
n_dups = dups.sum()
print(f"Duplicated episodes: {n_dups}")

if n_dups > 0:
    print(dfj.loc[dups, ['id_anon','Ingreso']].value_counts().head(10))

Duplicated episodes: 0


Se exploran los rangos mínimos y máximos de las fechas relevantes y se detectan valores sentinela (por ejemplo, 1753-01-01) utilizados para representar ausencia. Esto permite distinguir entre ausencia real y datos potencialmente corruptos.

In [7]:
# Block 6 — Date ranges and sentinel detection

def date_range(s):
    if s.notna().any():
        return s.min(), s.max(), s.notna().mean()*100
    return (pd.NaT, pd.NaT, 0.0)

print("=== Date ranges (min, max, non-null %) ===")
for c in ['FechaRegistro','FechaTriage','FechaPrimeraEvolucion','FechaPrimeraInterconsulta','FechaPrimeraEvaluacion','FechaEgreso']:
    if c in dfj.columns:
        dmin, dmax, cov = date_range(dfj[c])
        print(f"{c}: {dmin}  ->  {dmax}  | non-null: {cov:.2f}%")

# Sentinel check
sentinel = pd.Timestamp('1753-01-01 00:00:00')
for c in ['FechaEgreso','FechaPrimeraInterconsulta','FechaPrimeraEvolucion','FechaPrimeraEvaluacion']:
    if c in dfj.columns:
        n_sentinel = (dfj[c] == sentinel).sum()
        if n_sentinel:
            print(f"Sentinel {sentinel} in {c}: {n_sentinel} rows")

=== Date ranges (min, max, non-null %) ===
FechaRegistro: 2023-01-01 00:49:09  ->  2023-12-31 23:38:44  | non-null: 100.00%
FechaTriage: 2023-01-01 01:01:52  ->  2023-12-31 23:44:18  | non-null: 100.00%
FechaPrimeraEvolucion: 2023-01-01 01:09:52  ->  2024-01-13 18:08:45  | non-null: 99.95%
FechaPrimeraInterconsulta: 2023-01-01 03:00:07  ->  2024-01-01 12:54:39  | non-null: 51.24%
FechaPrimeraEvaluacion: 2023-01-01 01:09:52  ->  2024-01-13 18:08:45  | non-null: 100.00%
FechaEgreso: 1753-01-01 00:00:00  ->  2024-01-29 05:14:00  | non-null: 100.00%
Sentinel 1753-01-01 00:00:00 in FechaEgreso: 1135 rows


Se valida que los deltas de tiempo calculados sean coherentes y no negativos: registro→triage, triage→evaluación y registro→evaluación. Se reportan conteos y ejemplos si se encuentran anomalías.

In [8]:
# Block 7 — Time deltas sanity checks (non-negative, coherence)

checks = {
    'min_registro_a_triage': ('FechaRegistro','FechaTriage'),
    'min_triage_a_eval': ('FechaTriage','FechaPrimeraEvaluacion'),
    'min_registro_a_eval': ('FechaRegistro','FechaPrimeraEvaluacion')
}
for col, (a,b) in checks.items():
    if all(c in dfj.columns for c in [col, a, b]):
        neg = (dfj[col] < 0).sum()
        print(f"{col}: negatives = {neg}")
        if neg:
            bad = dfj.loc[dfj[col] < 0, [a,b,col]].head(5)
            print("Examples of negative deltas:\n", bad)

min_registro_a_triage: negatives = 0
min_triage_a_eval: negatives = 0
min_registro_a_eval: negatives = 0


Se listan frecuencias de variables categóricas y estadísticas de edad por ClasificacionTriage. Esto ayuda a confirmar que la estructura poblacional no cambió tras el join.

In [9]:
# Block 8 — Key distributions

if 'ClasificacionTriage' in dfj.columns:
    print("=== ClasificacionTriage distribution ===")
    print(dfj['ClasificacionTriage'].value_counts().to_frame('count').assign(
        proportion=lambda x: x['count']/x['count'].sum()
    ))

if 'SalidaClinica' in dfj.columns:
    print("\n=== SalidaClinica distribution ===")
    print(dfj['SalidaClinica'].value_counts())

if {'EdadAtencion','ClasificacionTriage'}.issubset(dfj.columns):
    g = dfj.groupby('ClasificacionTriage')['EdadAtencion']
    out = pd.DataFrame({
        'count': g.size(),
        'mean': g.mean().round(3),
        'median': g.median(),
        'min': g.min(),
        'max': g.max()
    })
    print("\n=== Age stats by ClasificacionTriage ===")
    print(out)


=== ClasificacionTriage distribution ===
                     count  proportion
ClasificacionTriage                   
3                    35635    0.772407
2                     9392    0.203576
4                      657    0.014241
1                      423    0.009169
5                       28    0.000607

=== SalidaClinica distribution ===
SalidaClinica
S    41777
N     4358
Name: count, dtype: int64

=== Age stats by ClasificacionTriage ===
                     count    mean  median  min  max
ClasificacionTriage                                 
1                      423  53.548    54.0   18   99
2                     9392  54.834    55.0   18  104
3                    35635  44.522    41.0   18  102
4                      657  38.735    37.0   18   86
5                       28  42.893    39.0   21   86


Se generan tablas de contingencia para ClasificacionTriage vs SalidaClinica y, opcionalmente, por género. Este cruce es útil para detectar desbalances inesperados.

In [10]:
# Block 9 — Cross tabs

if {'ClasificacionTriage','SalidaClinica'}.issubset(dfj.columns):
    ct = pd.crosstab(dfj['ClasificacionTriage'], dfj['SalidaClinica'], margins=True)
    print("=== Triage × SalidaClinica ===")
    print(ct)

if {'ClasificacionTriage','Genero'}.issubset(dfj.columns):
    cg = pd.crosstab(dfj['ClasificacionTriage'], dfj['Genero'], margins=True)
    print("\n=== Triage × Genero ===")
    print(cg)

=== Triage × SalidaClinica ===
SalidaClinica           N      S    All
ClasificacionTriage                    
1                      55    368    423
2                    1000   8392   9392
3                    3041  32594  35635
4                     243    414    657
5                      19      9     28
All                  4358  41777  46135

=== Triage × Genero ===
Genero               Femenino  Masculino    All
ClasificacionTriage                            
1                         169        254    423
2                        4943       4449   9392
3                       20268      15367  35635
4                         366        291    657
5                          12         16     28
All                     25758      20377  46135


Se presenta un resumen ejecutivo del control de calidad, incluyendo filas/columnas, nulos críticos, duplicados de episodios y rangos de fechas. Este resumen puede guardarse en un archivo de texto para auditoría.

In [11]:
# Block 10 — QA summary (print and optional save)

lines = []
lines.append(f"Rows: {len(dfj)} | Cols: {dfj.shape[1]}")
for c in ['FechaRegistro','FechaTriage','FechaPrimeraEvaluacion']:
    if c in dfj.columns:
        lines.append(f"{c} non-null: {dfj[c].notna().mean()*100:.2f}%")

# Duplicates
dup_count = dfj.duplicated(subset=['id_anon','Ingreso'], keep=False).sum() if {'id_anon','Ingreso'}.issubset(dfj.columns) else -1
lines.append(f"Duplicated episodes: {dup_count}")

# Date ranges
def dr(s):
    return (str(s.min()) if s.notna().any() else "NaT",
            str(s.max()) if s.notna().any() else "NaT")
for c in ['FechaRegistro','FechaTriage','FechaPrimeraEvaluacion','FechaEgreso']:
    if c in dfj.columns:
        mn, mx = dr(dfj[c])
        lines.append(f"{c} range: {mn} -> {mx}")

report = "\n".join(lines)
print("=== QA SUMMARY ===\n" + report)

# Optional save
OUT = r"C:\Users\wilmerbelza\Documents\Prediction model\artifacts\qa_summary.txt"
try:
    with open(OUT, "w", encoding="utf-8") as f:
        f.write(report)
    print(f"\nSaved QA summary to: {OUT}")
except Exception as e:
    print("Skip saving QA summary:", e)

=== QA SUMMARY ===
Rows: 46135 | Cols: 16
FechaRegistro non-null: 100.00%
FechaTriage non-null: 100.00%
FechaPrimeraEvaluacion non-null: 100.00%
Duplicated episodes: 0
FechaRegistro range: 2023-01-01 00:49:09 -> 2023-12-31 23:38:44
FechaTriage range: 2023-01-01 01:01:52 -> 2023-12-31 23:44:18
FechaPrimeraEvaluacion range: 2023-01-01 01:09:52 -> 2024-01-13 18:08:45
FechaEgreso range: 1753-01-01 00:00:00 -> 2024-01-29 05:14:00

Saved QA summary to: C:\Users\wilmerbelza\Documents\Prediction model\artifacts\qa_summary.txt
